In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import rasterio as rs
import rasterio.features as rsf
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gpd
from pyproj import CRS
# from sklearn.preprocessing import Normalize
from IPython.display import clear_output
import re
from tqdm import tqdm

Data Preprocessing: 
## Exporting Mosaicked and Masked Images 

In [ ]:
# MainF_str = input("Give the location of folder where S2A folders are stored: ")
# MainF_str = "Granule_Data_Banaskantha/"
MainF_str = "D:/Manav/Granule_Data_Banaskantha_23_24/"
MainF_files = os.listdir(MainF_str)
specDs = ["R10m"]
bands = ["B02","B03","B04","B08"]

# four sections of granules required for banaskantha district:
# BL -> Bottom Left - QYM
# BR -> Bottom Right - QZM
# UL -> Upper Left - RYN
# UR -> Upper Right - RZN

for j in tqdm(range(len(specDs))):
    specD = specDs[j]
    for k in tqdm(range(len(bands))):
        band = bands[k]
        if os.path.exists('Automate/Mosaicked_Data/'+specD+'_'+band) == False:
            os.makedirs('Automate/Mosaicked_Data/'+specD+'_'+band)
        
        # Step 1: Extracting location of data granules
        BL_loc_files = []
        BR_loc_files = []
        UL_loc_files = []
        UR_loc_files = []
        for i in range(len(MainF_files)):
            fname = MainF_files[i]
            if fname[41:44] == "QYM":
                SecFolder_str = MainF_str+fname+"/GRANULE/"
                SecFolder_File = os.listdir(SecFolder_str)
                SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(specD) != -1:
                        SecFolder_str = SecFolder_str+specD+"/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(band) != -1:
                        BL_loc_files.append(SecFolder_str+SecFolder_Files[j])
            if fname[41:44] == "QZM":
                SecFolder_str = MainF_str+fname+"/GRANULE/"
                SecFolder_File = os.listdir(SecFolder_str)
                SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(specD) != -1:
                        SecFolder_str = SecFolder_str+specD+"/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(band) != -1:
                        BR_loc_files.append(SecFolder_str+SecFolder_Files[j])
            if fname[41:44] == "RYN":
                SecFolder_str = MainF_str+fname+"/GRANULE/"
                SecFolder_File = os.listdir(SecFolder_str)
                SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(specD) != -1:
                        SecFolder_str = SecFolder_str+specD+"/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(band) != -1:
                        UL_loc_files.append(SecFolder_str+SecFolder_Files[j])
            if fname[41:44] == "RZN":
                SecFolder_str = MainF_str+fname+"/GRANULE/"
                SecFolder_File = os.listdir(SecFolder_str)
                SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(specD) != -1:
                        SecFolder_str = SecFolder_str+specD+"/"
                SecFolder_Files = os.listdir(SecFolder_str)
                for j in range(len(SecFolder_Files)):
                    if SecFolder_Files[j].find(band) != -1:
                        UR_loc_files.append(SecFolder_str+SecFolder_Files[j])

        # Arranging Chronologically
        date_pattern = r'(\d{4})(\d{2})(\d{2})'
        dates = [re.search(date_pattern, path).groups() for path in BL_loc_files]
        BL_loc_files = [path for _, path in sorted(zip(dates, BL_loc_files))]
        
        # Extracting Dates
        date_pattern = r'(\d{4})(\d{2})(\d{2})'
        dates = [re.search(date_pattern, path).groups() for path in BL_loc_files]
        dates_BL = []
        for i in range(len(dates)):
            tmp = dates[i][0]+dates[i][1]+dates[i][2]
            dates_BL.append(tmp)

        # Arranging Chronologically
        date_pattern = r'(\d{4})(\d{2})(\d{2})'
        dates = [re.search(date_pattern, path).groups() for path in BR_loc_files]
        BR_loc_files = [path for _, path in sorted(zip(dates, BR_loc_files))]

        date_pattern = r'(\d{4})(\d{2})(\d{2})'
        dates = [re.search(date_pattern, path).groups() for path in UL_loc_files]
        UL_loc_files = [path for _, path in sorted(zip(dates, UL_loc_files))]

        date_pattern = r'(\d{4})(\d{2})(\d{2})'
        dates = [re.search(date_pattern, path).groups() for path in UR_loc_files]
        UR_loc_files = [path for _, path in sorted(zip(dates, UR_loc_files))]


        # Step 2: Mosaicking Data Granules
        for i in tqdm(range(len(dates_BL)),desc="Mosaicking Granules"):
            tmp_img,tmp_transform = merge([BL_loc_files[i],BR_loc_files[i],UL_loc_files[i],UR_loc_files[i]])
            print("Mosaicking Completed")
            dst_crs = CRS.from_epsg(32642)
            with rs.open(
                'Automate/Mosaicked_Data/'+specD+'_'+band+'/Merged_'+dates_BL[i]+'_'+specD+'_'+band+'.tif',
                'w+',
                driver='GTiff',
                height=tmp_img.shape[1],
                width=tmp_img.shape[2],
                count=tmp_img.shape[0],
                dtype=np.int32,
                crs=dst_crs,
                transform=tmp_transform,
            ) as dest_file:
                dest_file.write(tmp_img)
            dest_file.close()
            clear_output(wait=True)
            print("Image Exported")
            print("Progress: ",(round((i+1)/(len(dates_BL))*100)))
        
        # Step 1: Masking Mosaicked Data with District + Agriculture Mask
        # Extracting location of Mosaicked Data
        MainFolder_str = "Automate/Mosaicked_Data/"
        MainFolder_files = os.listdir(MainFolder_str)
        loc_merged = []
        for i in range(len(MainFolder_files)):
            MainFolder_files[i] = MainFolder_str+MainFolder_files[i]
        for i in range(len(MainFolder_files)):
            loc_tmp = []
            Subfolder_files = os.listdir(MainFolder_files[i])
            for j in range(len(Subfolder_files)):
                loc_tmp.append(MainFolder_files[i]+"/"+Subfolder_files[j])
            loc_merged.append(loc_tmp)
        
        
#         loc_agrimask = input("Give the location of agriculture mask for masking process")
#         banasAgriShp = gpd.read_file(loc_agrimask)
        banasAgriShp = gpd.read_file("Shape Files/Final Shape Files/Banas_agriOnlyMask.shp")
        geometry = []
        for i in banasAgriShp['geometry']:
            geometry.append(i)
        
        if os.path.exists('Automate/Masked_Data/'+specD+'_'+band) == False:
            os.makedirs('Automate/Masked_Data/'+specD+'_'+band)
        
        #Masking and Stacking
        for i in tqdm(range(len(loc_merged)),desc="Main Loop"):
            tmp_stacked = []
            for j in tqdm(range(3,len(loc_merged[i])),desc="Sub Loop"):
                fname = loc_merged[i][j][-12:-4]
                tmp_r = rs.open(loc_merged[i][j],'r')
                tmp_data,tmp_transform = mask(dataset = tmp_r, shapes = geometry, crop=True)
                tmp_stacked.append(tmp_data[0])
                print("Masking Completed - "+str(fname)+", "+str(j))
                clear_output(wait=True)
            tmp_stacked = np.array(tmp_stacked)
            dst_crs = CRS.from_epsg(32642)
            with rs.open(
                'Automate/Masked_Data/'+fname+'/Masked_Stacked_1NOV_15MAR_'+fname+'.tif',
                'w',
                driver='GTiff',
                height=tmp_stacked.shape[1],
                width=tmp_stacked.shape[2],
                count=tmp_stacked.shape[0],
                dtype=np.float32,
                crs=dst_crs,
                transform=tmp_transform,
            ) as dest_file:
                dest_file.write(tmp_stacked)
            dest_file.close()
            print("Image Exported")

Data Preprocessing:
# Extracting Vegetation Indices

In [17]:
loc_band_files

[['Automate/Masked_Data/R10m_B03/Masked_Stacked_1NOV_15MARR10m_B03.tif'],
 ['Automate/Masked_Data/R10m_B04/Masked_Stacked_1NOV_15MARR10m_B04.tif'],
 ['Automate/Masked_Data/R10m_B08/Masked_Stacked_1NOV_15MARR10m_B08.tif']]

In [29]:
VI = ["EVI2","GNDVI","MSAVIhyper","MTVI1","NDMI","NDVI","OSAVI","PSRI","SAVI3","VARIgreen"]
specDs = ["R10m"]
bands = ["B02","B03","B04","B06","B08","B11"]
loc_band_files = []
for i in range(len(bands)):
    MainF_str = "Automate/Masked_Data/"
    MainF_files = os.listdir(MainF_str)
    loc_tmp = []
    for j in range(len(MainF_files)):
        PrimaryFolder_str = MainF_str
        if MainF_files[j].find(bands[i]) != -1:
            PrimaryFolder_str = PrimaryFolder_str+MainF_files[j]
            PrimaryFolder_files = os.listdir(PrimaryFolder_str)
            for k in range(len(PrimaryFolder_files)):
                loc_tmp.append(PrimaryFolder_str+"/"+PrimaryFolder_files[k])
    loc_band_files.append(loc_tmp)
    
# importing images into array
arr_band = []
for i in tqdm(range(len(loc_band_files))):
    arr_tmp = []
    for j in range(len(loc_band_files[i])):
        r = rs.open(loc_band_files[i][j])
        a = r.read()
        arr_tmp.append(a)
    arr_band.append(arr_tmp[0])
arr_band = np.array(arr_band)

# changing the range of image arrays
arr_band = arr_band/10000

# assigning indices to band
# Dictionary => band_dict = {"B02": 0, "B03":1, ....}
band_dict = {}
for i in range(len(bands)):
    band_dict[bands[i]] = i
    
# assigning numbers to VIs
# Dictionary => vi_dict = {"MTVI1": 0, "MSAVIhyper":1, ....}
vi_dict = {}
for i in range(len(VI)):
    vi_dict[VI[i]] = i

# generating mtvi and msavi arrays
arr_vi = []
for i in tqdm(range(len(VI))):
    arr_tmp = []
    if VI[i] == "MTVI1":
        arr_vi.append(1.2*((1.2*(arr_band[band_dict["B08"]]-arr_band[band_dict["B03"]]))-(2.5*(arr_band[band_dict["B04"]]-arr_band[band_dict["B03"]]))))
    elif VI[i] == "MSAVIhyper":
        arr_vi.append(0.5 * (((2 * arr_band[band_dict["B08"]]) + 1) - (np.sqrt((2 * arr_band[band_dict["B08"]] + 1)**2 + 8 * (arr_band[band_dict["B08"]] - arr_band[band_dict["B04"]])))))
    elif VI[i] == "NDVI":
        arr_vi.append((arr_band[band_dict["B08"]] - arr_band[band_dict["B04"]])/(arr_band[band_dict["B08"]] + arr_band[band_dict["B04"]]))
    elif VI[i] == "EVI2":
        arr_vi.append(2.5*((arr_band[band_dict["B08"]]-arr_band[band_dict["B04"]])/(arr_band[band_dict["B08"]]+(6*arr_band[band_dict["B04"]])+1)))
    elif VI[i] == "GNDVI":
        arr_vi.append((arr_band[band_dict["B08"]]-arr_band[band_dict["B03"]])/(arr_band[band_dict["B08"]]+arr_band[band_dict["B03"]]))
    elif VI[i] == "OSAVI":
        arr_vi.append(1.16*(arr_band[band_dict["B08"]]-arr_band[band_dict["B04"]])/(arr_band[band_dict["B08"]]+arr_band[band_dict["B04"]]+0.16))
    elif VI[i] == "PSRI":
        arr_vi.append((arr_band[band_dict["B04"]] - arr_band[band_dict["B02"]])/arr_band[band_dict["B06"]])
    elif VI[i] == "SAVI3":
        arr_vi.append(1.5*(arr_band[band_dict["B08"]] - arr_band[band_dict["B04"]])/(arr_band[band_dict["B08"]] + arr_band[band_dict["B04"]]+0.5))
    elif VI[i] == "VARIgreen":
        arr_vi.append((arr_band[band_dict["B03"]]-arr_band[band_dict["B04"]])/(arr_band[band_dict["B03"]]+arr_band[band_dict["B04"]]+arr_band[band_dict["B02"]]))
    elif VI[i] == "NDMI":
        arr_vi.append((arr_band[band_dict["B08"]] - arr_band[band_dict["B11"]])/(arr_band[band_dict["B08"]] + arr_band[band_dict["B11"]]))
arr_vi = np.array(arr_vi,dtype=np.float32)

for i in range(len(VI)):
    if os.path.exists('Automate/Vegetation_Indices/'+VI[i]+'_R10m/') == False:
        os.makedirs('Automate/Vegetation_Indices/'+VI[i]+'_R10m/')

r_export = rs.open(loc_band_files[0][0])
transform = r_export.profile['transform']            
for i in tqdm(range(len(VI))): 
    dst_crs = CRS.from_epsg(32642)
    with rs.open(
        'Automate/Vegetation_Indices/'+VI[i]+'_R10m/'+VI[i]+'_Stacked_R10m_1NOV_15MAR.tif',
        'w',
        driver='GTiff',
        height=arr_vi[i].shape[1],
        width=arr_vi[i].shape[2],
        count=arr_vi[i].shape[0],
        dtype=np.float32,
        crs=dst_crs,
        transform=transform,
    ) as dest_file:
        dest_file.write(arr_vi[i])
    dest_file.close()

 50%|█████████████████████████████████████████▌                                         | 1/2 [02:19<02:19, 139.06s/it]C:\Users\admin\AppData\Local\Temp\ipykernel_7784\2848005634.py:51: RuntimeWarning: invalid value encountered in sqrt
  arr_vi.append(0.5 * (((2 * arr_band[band_dict["B08"]]) + 1) - (np.sqrt((2 * arr_band[band_dict["B08"]] + 1)**2 + 8 * (arr_band[band_dict["B08"]] - arr_band[band_dict["B04"]])))))
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [04:17<00:00, 128.52s/it]


## Detecting the range of dates for select bands and VIs

Importing VIs

In [2]:
VI = ["MTVI1","MSAVIhyper"]
arr_vi = []
for i in range(len(VI)):
    r_tmp = rs.open('Automate/Vegetation_Indices/'+VI[i]+'_R10m/'+VI[i]+'_Stacked_R10m_1NOV_15MAR.tif')
    arr_tmp = r_tmp.read()
    arr_vi.append(arr_tmp)
arr_vi = np.array(arr_vi,dtype=np.float32)

In [7]:
potato_pixel = []
VI = ["MTVI1","MSAVIhyper"]
for i in range(2):
    if i==1: 
        tmp = input("Give X pixel coordinate to check the trend of potato time series: ")
    else:
        tmp = input("Give Y pixel coordinate to check the trend of potato time series: ")
    potato_pixel.append(int(tmp))
vals_arr = []
vals_arr.append(arr_vi[0][:,potato_pixel[0],potato_pixel[1]])
vals_arr.append(arr_vi[1][:,potato_pixel[0],potato_pixel[1]])
mtvi_select_diff = [i for i in range(np.argmax(vals_arr[0])+1)]
msavi_select_pg = np.argmin(vals_arr[1])
mtvi_select_h = np.argmax(vals_arr[0]) + np.argmin(vals_arr[0][(np.argmax(vals_arr[0])):])
mtvi_select_h = 9
print(mtvi_select_diff,mtvi_select_h,msavi_select_pg)

Give Y pixel coordinate to check the trend of potato time series: 3845
Give X pixel coordinate to check the trend of potato time series: 11689
[0, 1, 2, 3, 4, 5, 6] 9 6


In [8]:
mtvi_diff = np.zeros((arr_vi.shape[2],arr_vi.shape[3]))
for i in tqdm(range(arr_vi.shape[1]-1)):
    tmp_arr_t_1 = np.array(arr_vi[0][i+1])
    tmp_arr_t = np.array(arr_vi[0][i])
    mtvi_diff = mtvi_diff + (abs(tmp_arr_t_1 - tmp_arr_t))**2
mtvi_diff = mtvi_diff/arr_vi.shape[1]
mtvi_pg = arr_vi[0][mtvi_select_diff[-1]]
mtvi_h = arr_vi[0][mtvi_select_h]
msavi_pg = arr_vi[1][msavi_select_pg]

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:28<00:00,  2.39s/it]


## Unsupervised Processing

In [9]:
p_feature = (mtvi_diff * mtvi_pg)/(mtvi_h * msavi_pg)

C:\Users\admin\AppData\Local\Temp\ipykernel_4564\2011044499.py:1: RuntimeWarning: divide by zero encountered in divide
  p_feature = (mtvi_diff * mtvi_pg)/(mtvi_h * msavi_pg)
C:\Users\admin\AppData\Local\Temp\ipykernel_4564\2011044499.py:1: RuntimeWarning: invalid value encountered in divide
  p_feature = (mtvi_diff * mtvi_pg)/(mtvi_h * msavi_pg)


In [10]:
threshold = -0.2078
segmented_output = np.full(p_feature.shape,np.nan,dtype=np.float32)
for i in tqdm(range(p_feature.shape[0])):
    for j in range(p_feature.shape[1]):
        if np.isnan(p_feature[i,j]) == False:
            if p_feature[i,j] < threshold:
                segmented_output[i,j] = 1
            else:
                segmented_output[i,j] = 0

100%|██████████████████████████████████████████████████████████████████████████████| 9877/9877 [07:41<00:00, 21.42it/s]


Displaying Output

In [ ]:
# Manual Latitude Longitude
lat1, lon1 = 23.84394, 71.20126 
lat2, lon2 = 24.27392, 73.02098  
x_points = np.linspace(lat1, lat2, segmented_output.shape[1])
y_points = np.linspace(lon1, lon2, segmented_output.shape[0])
y_points= y_points[::-1]

In [ ]:
labels = ['Others','Potato','Other Crops']
values = np.unique(segmented_output.ravel())
values = values[::-1]
N = len(labels)
clusters = [i+1 for i in range(N)]
fig = plt.figure(figsize=(15,10))
img = plt.imshow(segmented_output)
colors = [img.cmap(img.norm(value)) for value in values]
patches = [mpatches.Patch(color=colors[i], label="Crop: {0}".format(labels[i]), edgecolor="black", linewidth=1) for i in range(len(values)) ]
ax = plt.gca()
ax.tick_params(axis='x', labelsize=22)
ax.tick_params(axis='y', labelsize=22)

x_ticks_interval = 3000
y_ticks_interval = 3000
x_ticks = np.arange(0, len(x_points), x_ticks_interval)
y_ticks = np.arange(0, len(y_points), y_ticks_interval)
ax.set_xticks(x_ticks)
ax.set_yticks(y_ticks)
ax.set_xticklabels([f'{x_points[i]:.2f}' for i in x_ticks])
ax.set_yticklabels([f'{y_points[i]:.4f}' for i in y_ticks])

compass_img = plt.imread('compass.png')
imagebox = OffsetImage(compass_img, zoom=0.07)
ab = AnnotationBbox(imagebox, (0.93, 0.12), xycoords='axes fraction', frameon=False, boxcoords="axes fraction", pad=0)
ax.add_artist(ab)

t = input("Give a title: ")
plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0, prop={'size': 'xx-large'})
plt.title(t, fontdict={"size":27}, pad=30, weight="bold")
plt.tight_layout()

if os.path.exists('Automate/Outputs/') == False:
    os.makedirs('Automate/Outputs/')

plt.savefig("Automate/Outputs/"+t+".png", dpi=300)
plt.show()

## Supervised Processing

Training Data Preparation

In [5]:
import pandas as pd
import sklearn
import os
import rasterio as rs
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier
from sklearn.metrics import accuracy_score,balanced_accuracy_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from IPython.display import clear_output
import matplotlib.pyplot as plt
import concurrent.futures
from tqdm import tqdm
import re
import logging

from rasterio.plot import show
from rasterio.merge import merge
import geopandas as gpd
from rasterio.features import geometry_mask
from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import polygonize
import fiona
from pyproj import CRS
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
import warnings
warnings.filterwarnings('ignore')

In [15]:
# Extract Dates
MainF_str = "D:/Manav/Granule_Data_Banaskantha_23_24/"
MainF_files = os.listdir(MainF_str)
specD = "R10m"

band = "B03"
BL_loc_files = []
BR_loc_files = []
UL_loc_files = []
UR_loc_files = []
for i in range(len(MainF_files)):
    fname = MainF_files[i]
    if fname[41:44] == "QYM":
        SecFolder_str = MainF_str+fname+"/GRANULE/"
        SecFolder_File = os.listdir(SecFolder_str)
        SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(specD) != -1:
                SecFolder_str = SecFolder_str+specD+"/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(band) != -1:
                BL_loc_files.append(SecFolder_str+SecFolder_Files[j])
    if fname[41:44] == "QZM":
        SecFolder_str = MainF_str+fname+"/GRANULE/"
        SecFolder_File = os.listdir(SecFolder_str)
        SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(specD) != -1:
                SecFolder_str = SecFolder_str+specD+"/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(band) != -1:
                BR_loc_files.append(SecFolder_str+SecFolder_Files[j])
    if fname[41:44] == "RYN":
        SecFolder_str = MainF_str+fname+"/GRANULE/"
        SecFolder_File = os.listdir(SecFolder_str)
        SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(specD) != -1:
                SecFolder_str = SecFolder_str+specD+"/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(band) != -1:
                UL_loc_files.append(SecFolder_str+SecFolder_Files[j])
    if fname[41:44] == "RZN":
        SecFolder_str = MainF_str+fname+"/GRANULE/"
        SecFolder_File = os.listdir(SecFolder_str)
        SecFolder_str = SecFolder_str+SecFolder_File[0]+"/IMG_DATA/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(specD) != -1:
                SecFolder_str = SecFolder_str+specD+"/"
        SecFolder_Files = os.listdir(SecFolder_str)
        for j in range(len(SecFolder_Files)):
            if SecFolder_Files[j].find(band) != -1:
                UR_loc_files.append(SecFolder_str+SecFolder_Files[j])

# Arranging Chronologically
date_pattern = r'(\d{4})(\d{2})(\d{2})'
dates = [re.search(date_pattern, path).groups() for path in BL_loc_files]
BL_loc_files = [path for _, path in sorted(zip(dates, BL_loc_files))]

# Extracting Dates
date_pattern = r'(\d{4})(\d{2})(\d{2})'
dates = [re.search(date_pattern, path).groups() for path in BL_loc_files]
dates_BL = []
for i in range(len(dates)):
    tmp = dates[i][0]+dates[i][1]+dates[i][2]
    dates_BL.append(tmp)
dates_BL = dates_BL[3:]
print(dates_BL)

['20231101', '20231111', '20231121', '20231201', '20231216', '20231226', '20240110', '20240120', '20240209', '20240219', '20240229', '20240305', '20240315']


In [6]:
#first gt
loc_gt = input("Location of Ground truth: ")
gt = gpd.read_file(loc_gt)
crs_input = input("CRS for the GT: ")
dst_crs = CRS.from_epsg(crs_input)
gt = gt.to_crs(dst_crs)
label_colname = input("Give the name of crop name column: ")
gt_1 = gt[['geometry',label_colname]]
geometry = []
croplabel = []
for i in range(len(gt_1)):  
    geometry.append(gt_1['geometry'][i])
    croplabel.append(gt_1[label_colname][i])
    
# second gt
loc_gt1 = input("Location of Ground Truth: ")
gt1 = gpd.read_file(loc_gt1)
gt1 = gt1.drop(gt1[gt1['Banas_IS_3']=='Non-Crop-Area'].index)
gt1 = gt1.to_crs(dst_crs)
label_colname1 = input("Give the name of crop name column: ")
gt1_1 = gt1[['geometry',label_colname1]]
gt1_1 = gt1_1.reset_index()
gt1_1 = gt1_1.drop(columns=['index'])

geometry1 = []
croplabel1 = []
for i in range(len(gt1_1)):  
    geometry1.append(gt1_1['geometry'][i])
    croplabel1.append(gt1_1['Banas_IS_4'][i])
    
print("Ground Truth Processed. Reading Image Data for Training Data Preparation Now.")
r = rs.open("Automate/Masked_Data/R10m_B03/Masked_Stacked_1NOV_15MARR10m_B03.tif")
raster_data = np.array(r.read()[0])
transform = r.transform
coordinates = pd.DataFrame(columns=['label','pixels'])
i = 0
for index, row in gt_1.iterrows():
    polygon = row.geometry
    label = row[label_colname]  
    
    mask = geometry_mask([polygon], out_shape=raster_data.shape, transform=transform, invert=True)
    coords = np.argwhere(mask)
    
    coordinates.loc[len(coordinates.index)] = [label,coords]
    print("Progress: ",(round(i/len(gt_1)*100,4)))
    clear_output(wait=True)
    i=i+1
    
i = 0
for index, row in gt1_1.iterrows():
    polygon = row.geometry
    label = row[label_colname1]
    
    mask = geometry_mask([polygon], out_shape=raster_data.shape, transform=transform, invert=True)
    coords = np.argwhere(mask)
    
    coordinates.loc[len(coordinates.index)] = [label,coords]
    print("Progress: ",(round(i/len(gt1_1)*100,4)))
    clear_output(wait=True)
    i=i+1
    
if os.path.exists('Automate/Outputs/') == False:
    os.makedirs('Automate/Outputs/')
    
coordinates.to_csv("Automate/Outputs/pixels_labels.csv",index=False)

df = pd.read_csv("Automate/Outputs/pixels_labels.csv")
print(df.head())
df1 = pd.DataFrame(columns=["label","pixels"])

# New DataFrame with converted pixel coordinates from string to integer
for i in range(len(df)):
    output = df['pixels'][i]
    numbers = re.findall(r'\d+', output)
    arrays = np.array(numbers).reshape(-1, 2).astype(int)
    df1.loc[len(df1.index)] = [df['label'][i],arrays]
df1 = df1.drop(len(df1)-1)
labels = df1['label'].unique()

# Removing Invalid Crops
df1 = df1[~df1['label'].isin(['Groundnut','Napier-Grasses','Chikori','Sorghum-Jowar','Pomegranate','Rajgaro','Gram','Tobacco','Cotton'])]

# Removing invalid pixels
df1 = df1.reset_index()
for i in range(len(df1)):
    if len(df1['pixels'][i]) == 0:
        df1 = df1.drop(i)
df1 = df1.reset_index()

VI = ["MTVI1","MSAVIhyper"]
arr_vi = []
print("Importing Vegetation Indices")
for i in range(len(VI)):
    r_tmp = rs.open('Automate/Vegetation_Indices/'+VI[i]+'_R10m/'+VI[i]+'_Stacked_R10m_1NOV_15MAR.tif')
    arr_tmp = r_tmp.read()
    arr_vi.append(arr_tmp)
arr_vi = np.array(arr_vi,dtype=np.float32)

print("Exporting Training Data")
for k in range(len(VI)):
    df_data = pd.DataFrame(columns=['coordinates']+dates_BL+['labels'])
    for i in tqdm(range(len(df1))):
        coordinates = df1['pixels'][i]
        label = df1['label'][i]
        for j in coordinates:
            values = arr_vi[k][:,j[0],j[1]]
            tmp = [j]+list(values)+[label]
            df_data.loc[len(df_data.index)] = tmp
    df_data.to_csv("Automate/Outputs/Training_data_final_"+VI[k]+".csv",index=False)

   label                                             pixels
0  Maize  [[ 5993 15318]\n [ 5993 15319]\n [ 5994 15317]...
1  Maize  [[ 6088 15129]\n [ 6089 15129]\n [ 6090 15129]...
2  Maize  [[ 6267 15229]\n [ 6267 15230]\n [ 6268 15227]...
3  Maize  [[ 6028 15138]\n [ 6028 15139]\n [ 6029 15138]...
4  Maize  [[ 5987 15136]\n [ 5987 15137]\n [ 5988 15136]...


100%|████████████████████████████████████████████████████████████████████████████████| 286/286 [00:22<00:00, 12.64it/s]


In [ ]:
# Model Training on a single training data
loc_train = input("Enter location of training data: ")
# df = pd.read_csv("Automate/Outputs/Training_data_final_MTVI1.csv")

# extracting VI name
mark = -1
for i in range(len(loc_train)-1,-1,-1):
    if loc_train[i] == "_":
        mark = i
        break
VI_name = loc_train[mark+1:-4]

df = pd.read_csv(loc_train)
df = df.dropna()
df = df.drop(columns=[df.columns[0]])
label_map = {'Maize':1, 'Wheat':2, 'Mustard':3, 'castor':4, 'Potato':8, 'Musturd':3,
       'Castor':4, 'Oat':5, 'Cumin':6, 'Lucerne':7}
df['labels_num'] = df['labels'].map(label_map)
dates_BL = df.columns[:-2]
train_x, test_x, train_y, test_y = train_test_split(df[dates_BL],df['labels_num'],random_state=42,test_size=0.25)

choice = input("Enter which classifier (Enter Serial Number):\n1. RF\n2. XGBoost\n")
classifier = ""
if choice == '1':
    # Random Forest Classifier
    clf = RandomForestClassifier(n_estimators=25)
    classifier = "RF"
    clf.fit(train_x,train_y)
elif choice == '2':
    #XGBoost Classifier
    clf = XGBClassifier()
    classifier = "XGB"
    clf.fit(train_x,train_y)

predict_y = clf.predict(test_x)
print("Accuracy: {0} %".format(accuracy_score(test_y,predict_y)*100))
print(classification_report(test_y,predict_y))

VI = ["MTVI1","MSAVIhyper"]
arr_vi = []
print("Importing Vegetation Indices")
for i in range(len(VI)):
    r_tmp = rs.open('Automate/Vegetation_Indices/'+VI[i]+'_R10m/'+VI[i]+'_Stacked_R10m_1NOV_15MAR.tif')
    arr_tmp = r_tmp.read()
    arr_vi.append(arr_tmp)
arr_vi = np.array(arr_vi,dtype=np.float32)
VI_dict = {}
for i in range(len(VI)):
    VI_dict[VI[i]] = i

print("Predicting Using "+classifier)
valid_pixels = np.argwhere(arr_vi[0][0]!=0)
values_rows = []
def pixel_values(arr):
    values = arr_vi[VI_dict[VI_name]][:,arr[0],arr[1]]
    tmp = [[arr[0],arr[1]]]+list(values) 
    return tmp   

for i in tqdm(range(len(valid_pixels)), desc="Processing"):
    values_rows.append(pixel_values(valid_pixels[i]))
    
# prediction
prediction_y = np.full((arr_vi[VI_dict[VI_name]].shape[1],arr_vi[VI_dict[VI_name]].shape[2]),np.nan,dtype=np.float32)

def predict_clf(arr):
    return clf.predict(arr)[0]

for i in tqdm(range(len(values_rows)), desc="Processing"):
    values = values_rows[i][1:]
    values = np.reshape(values,(1,-1))
    prediction_y[values_rows[i][0][0],values_rows[i][0][1]] = predict_clf(values)
    
r = rs.open("Automate/Masked_Data/R10m_B03/Masked_Stacked_1NOV_15MARR10m_B03.tif")
transform = r.profile['transform']
print(transform)

if os.path.exists('Automate/Outputs/') == False:
    os.makedirs('Automate/Outputs/')

dst_crs = CRS.from_epsg(32642)
with rs.open(
    'Automate/Outputs/Segmented_'+VI+'_'+classifier+'.tif',
    'w',
    driver='GTiff',
    height=prediction_y.shape[0],
    width=prediction_y.shape[1],
    count=1,
    dtype=np.int16,
    crs=dst_crs,
    transform=transform,
) as dest_file:
    dest_file.write(prediction_y,1)
dest_file.close()